Run this notebook to reduce data from the Nickel Telescope. Be sure to read the notes in between the cells.

In [2]:
import numpy as np
import pandas as pd
import os
from astropy.io import fits
from pathlib import Path

from overscan_subtraction import overscan_subtraction
from bias_subtraction import bias_subtraction
from dark_subtraction import dark_subtraction
from flat_division import flat_division
from correct_object_name import correct_object_name

The reduction requires the OBJECT names in the raw FITS files (which are set at the time of the observation) to be one of the following:
- "bias"
- "dark"
- "dome flat"
- "sky flat" (i.e., flats taken of the sky at sunset)
- "focus"
- your target names (can be anything)

If you need to correct OBJECT an in FITS headers of any files, do that below. If everything is correct, ignore the next cell.

In [3]:
# correct_object(myfiles, "dark")

Now you're ready to reduce your data. First, get a list of all the raw data files. The name of the directory with the raw data should have format 'YYYY-MM-DD-nickel-raw'.

In [4]:
rawdir = Path("C:/Users/allis/Documents/2024-2025_Local/Akamai Internship/pipeline-testing/test-data-05-12/raw")  # path to directory with raw data
rawfiles = [file for file in rawdir.iterdir() if file.is_file()]



# rawfiles = [rawdir + '/' + file for file in sorted(os.listdir(rawdir)) if os.path.isfile(os.path.join(rawdir, file))]
# rawfiles 

Create processing and reduced directories.

In [5]:
rootdir = rawdir.parent
procdir = rawdir.parent / (rawdir.name + "-proc")
reddir = rawdir.parent / (rawdir.name + "-reduced")

# print(procdir)
# print(reddir)

Path.mkdir(procdir, exist_ok=True)
Path.mkdir(reddir, exist_ok=True)

# rawdir_split = rawdir.split("/")
# if rawdir_split[-1] == "":
#     rootdir = "/".join(rawdir_split[:-2])+"/"
#     datadir = rawdir_split[-2]+"/"
# else:
#     rootdir = "/".join(rawdir_split[:-1])+"/"
#     datadir = rawdir_split[-1]+"/"   

# datadir_split = datadir.split("-")

# procdir = "-".join(datadir_split[:4])+"-proc/"
# reddir = "-".join(datadir_split[:4])+"-red/"

# print(procdir)
# print(reddir)

# if not os.path.exists(procdir):
#     os.makedirs(procdir)
# if not os.path.exists(reddir):
#     os.makedirs(reddir)

Do overscan subtraction.

In [6]:
overscan_dir = procdir / 'overscan'
print(overscan_dir)
Path.mkdir(overscan_dir, exist_ok=True)
overscan_files = [overscan_dir / (file.stem + '_proc_over' + file.suffix) for file in rawfiles]
# print(overscan_files)

overscan_subtraction(rawfiles, overscan_files, 'yes')

# procdir = rootdir+procdir
# [procdir path] + [file name] + [_proc.] + 'fits' (for all files, excluding subdirectories)
# procfiles = [procdir + file.split('.')[0] + '_proc.' + file.split('.')[1] 
#              for file in sorted(os.listdir(rawdir)) 
#              if os.path.isfile(os.path.join(rawdir, file))]
# print(procfiles)

C:\Users\allis\Documents\2024-2025_Local\Akamai Internship\pipeline-testing\test-data-05-12\raw-proc\overscan


Make a dataframe of all the files we want to continue reducing.

In [7]:
obj_list = []
exptime_list = []
filt_list = []

for overscan_file in overscan_files:
    hdul = fits.open(str(overscan_file))
    obj_list.append(hdul[0].header["OBJECT"])
    exptime_list.append(hdul[0].header["EXPTIME"])
    filt_list.append(hdul[0].header["FILTNAM"])
    hdul.close()

df_log = pd.DataFrame({
    "file": overscan_files,
    "object": obj_list,
    "exptime": exptime_list,
    "filt": filt_list
    })
df_log

,file,object,exptime,filt
0,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
1,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
2,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
3,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
4,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
...,...,...,...,...
80,C:\Users\allis\Documents\2024-2025_Local\Akama...,PG1530+057,60,I
81,C:\Users\allis\Documents\2024-2025_Local\Akama...,PG1530+057,60,I
82,C:\Users\allis\Documents\2024-2025_Local\Akama...,PG1530+057,60,I
83,C:\Users\allis\Documents\2024-2025_Local\Akama...,PG1530+057,60,I


Do bias subtraction.

In [8]:
# gather all the bias frames
biasfiles = list(df_log.file[df_log.object == 'bias'])

# average all of them into one
biasdata = []
for biasfile in biasfiles:
    hdul = fits.open(str(biasfile))
    biasdata.append(hdul[0].data)
    hdul.close()
bias = np.stack(biasdata).mean(axis=0)

################ what is this? ###################
# omit hot column so that it is properly flat-fielded out
bias[:,256] = 0

# gather all non-bias files in a subdirectory of -proc
nonbias_dir = procdir / 'unbias'
print(nonbias_dir)
Path.mkdir(nonbias_dir, exist_ok=True)

nonbias_files_input = list(df_log.file[df_log.object != 'bias'])
nonbias_files_output = [nonbias_dir / (file.stem + '_proc_unbias' + file.suffix) for file in nonbias_files_input]


# file1 = nonbias_files_input[0]
# print(os.path.basename(nonbias_files_input[0]))
# dirname, filename = os.path.split(file1)
# comp1 = filename.split('_')
# comp1[1] = 'bias'+comp1[1]
# newfile = os.path.join(dirname, "_".join(comp1))
# print(newfile)

# nonbias_files_output = [newfile] + nonbias_files_input[1:]

bias_subtraction(nonbias_files_input, nonbias_files_output, bias)

df_log["file"] = [nonbias_dir / (file.stem + '_proc_unbias' + file.suffix) for file in df_log["file"]]
df_log

C:\Users\allis\Documents\2024-2025_Local\Akamai Internship\pipeline-testing\test-data-05-12\raw-proc\unbias


,file,object,exptime,filt
0,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
1,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
2,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
3,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
4,C:\Users\allis\Documents\2024-2025_Local\Akama...,bias,0,B
...,...,...,...,...
80,C:\Users\allis\Documents\2024-2025_Local\Akama...,PG1530+057,60,I
81,C:\Users\allis\Documents\2024-2025_Local\Akama...,PG1530+057,60,I
82,C:\Users\allis\Documents\2024-2025_Local\Akama...,PG1530+057,60,I
83,C:\Users\allis\Documents\2024-2025_Local\Akama...,PG1530+057,60,I


Delete all overscan subtracted files to save memory

In [9]:
# for file in overscan_files:
#     file.unlink()

Do dark subtraction.

This can usually be skipped, since the Nickel CCD has a very low dark current, but it is included here for the sake of completeness.

In [10]:
# if 'dark' in list(set(obj_list)):
#     darkexptimes = list(set(df_log.exptime[df_log.object == 'dark']))
#     for darkexptime in darkexptimes:
#         # find all files with this exposure time
#         darkfiles = list(df_log.file[(df_log.object == 'dark') & (df_log.exptime == darkexptime)])
#         flatfiles = list(df_log.file[((df_log.object == 'dome flat') | (df_log.object == 'sky flat')) &
#                                      (df_log.exptime == darkexptime)])
#         sciencefiles = list(df_log.file[(df_log.object != 'bias') &
#                                         (df_log.object != 'dark') &
#                                         (df_log.object != 'dome flat') &
#                                         (df_log.object != 'sky flat') &
#                                         (df_log.object != 'focus') &
#                                         (df_log.exptime == darkexptime)])
        
#         # calculate average dark frame
#         if len(darkfiles) > 1:
#             darkdata = []
#             for darkfile in darkfiles:
#                 hdul = fits.open(darkfile)
#                 darkdata.append(hdul[0].data)
#                 hdul.close()
#             dark = np.stack(darkdata).mean(axis=0)
#         else:
#             hdul = fits.open(darkfile)
#             dark = hdul[0].data
#             hdul.close()
        
#         # do dark subtraction
#         if len(flatfiles) > 0:
#             dark_subtraction(flatfiles, flatfiles, dark)
#         if len(sciencefiles) > 0:
#             dark_subtraction(sciencefiles, sciencefiles, dark)

# else:
#     print('No dark frames detected. Skipping dark subtraction.')

Do flat division.

Divide each pixel and then multiply all pixels by the average of the flat frame.

If sky (sunset) flats are available, those are used. If they are not available, dome flats are used.

In [11]:
# use sky flats if available, use dome flats if not
if 'sky flat' in list(set(obj_list)):
    flattype = 'sky flat'
else:
    flattype = 'flat'   # modified from 'dome flat' to 'flat' since 05-12-24 data uses different names
    
flatfilts = list(set(df_log.filt[df_log.object == flattype]))

for flatfilt in flatfilts:
    # find all the files with this filter
    flatfiles = list(df_log.file[(df_log.object == flattype) & (df_log.filt == flatfilt)])
    scienceobjects = list(set(df_log.object[(df_log.object != 'bias') &
                                            (df_log.object != 'dark') &
                                            (df_log.object != 'flat') &    # modified from 'dome flat' to 'flat' for 05-12-24 data
                                            (df_log.object != 'sky flat') &
                                            (df_log.object != 'focus') &
                                            (df_log.filt == flatfilt)]))
    
    # calculate the average flat frame
    if len(flatfiles) > 1:
        flatdata = []
        for flatfile in flatfiles:
            hdul = fits.open(str(flatfile))
            flatdata.append(hdul[0].data)
            hdul.close()
        print(flatdata)
        flat = np.stack(flatdata).mean(axis=0)
    else:
        hdul = fits.open(str(flatfile))
        flat = hdul[0].data
        hdul.close()
        
    if len(scienceobjects) > 0:
        for scienceobject in scienceobjects:
            sciencefiles = list(df_log.file[(df_log.object == scienceobject) &
                                            (df_log.filt == flatfilt)])
            
            # make a new directory for each science target / filter combination
            sci_dir = reddir / (scienceobject + '_' + flatfilt)
            Path.mkdir(sci_dir, exist_ok=True)
            # os.makedirs(rootdir+reddir+thisdir)

            # define reduced file names
            redfiles = [sci_dir / (file.stem.split('_')[0] + '_red' + file.suffix) for file in sciencefiles]
            
            # short_sciencefiles = [file.split('/')[-1] for file in sciencefiles]
            # framenum = [frame.split('_')[0] for frame in short_sciencefiles]
            # redfiles = [rootdir + reddir + thisdir + frame + '_red.fits' for frame in framenum]

            # do flat division
            if len(sciencefiles) > 0:
                flat_division(sciencefiles, redfiles, flat)

[array([[  943.92957,   627.42957,   712.72955, ..., 15962.13   ,
        16296.7295 , 28274.83   ],
       [ 2668.6384 ,  1505.0385 ,  1501.3384 , ..., 35980.438  ,
        37834.54   , 64598.438  ],
       [ 3213.3472 ,  1665.5471 ,  1623.4471 , ..., 36943.645  ,
        38954.047  , 64602.746  ],
       ...,
       [ 1222.0947 ,   569.7948 ,   514.9948 , ..., 10526.694  ,
         8934.995  , 14405.295  ],
       [ 1167.6449 ,   526.3449 ,   543.3449 , ...,  8339.645  ,
         6999.845  , 10711.345  ],
       [ 1270.5951 ,   702.89514,   603.1951 , ...,  6892.7954 ,
         5866.8955 ,  7835.8955 ]], dtype='>f4'), array([[ 1108.5116 ,   717.01154,   809.3115 , ..., 18332.71   ,
        18811.312  , 32446.412  ],
       [ 3069.2227 ,  1757.6228 ,  1804.9227 , ..., 42110.023  ,
        44005.125  , 64593.023  ],
       [ 3617.9338 ,  1865.1338 ,  1856.0338 , ..., 42978.23   ,
        44775.633  , 64597.332  ],
       ...,
       [ 1366.2384 ,   641.9384 ,   585.1384 , ..., 11999.83

: 

You're done! Your reduced images are now ready for your viewing.